<a href="https://colab.research.google.com/github/boriskuchin/MADMO-BASE-2024/blob/main/12_decision_trees_random_forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Решающие деревья. Случайный лес

## Введение

### Простое определение

**Дерево принятия решений (decision tree)** - алгоритм машинного обучения, предсказывает целевую переменную с помощью применения последовательности простых решающих правил (предикатов).


### Наглядный пример

![](https://i.kym-cdn.com/entries/icons/original/000/006/117/81939832_3194411770585244_1699876574616092672_n.png)

### Менее наглядный пример

![](https://yastatic.net/s3/ml-handbook/admin/3_2_41c1793bef.png)

### Более строгое определение

**Решающее дерево** - бинарное дерево, в котором:
1. каждой внутренней вершине $v$ приписан предикат $B_v: \mathbb{X} \to \{0,1\}$
2. каждой листовой вершине $v$ приписан прогноз $c_v \in \mathbb{Y}$, где $\mathbb{Y}$ — область значений целевой переменной (в случае классификации листу может быть также приписан вектор вероятностей классов).

**Предсказание** - проход по этому дереву от корня к листу такой, что в каждой встречаемой внутренней вершине $v$ считаем значение $B_v(x)$: если $B_v(x)=1$, то идем вправо, если $B_v(x)=0$, то идем влево.

### Структура предиката

Вообще предикат $B_v(x)$ может иметь любую структуру, однако, на практике чаще всего используют сравнение j-го признака с некоторым пороговым значением:

$$B_v(x, j, t) = [x_j \leq t]$$

Из структуры предиката можем сделать следующие выводы:

1. выученная функция является кусочно-постоянной, из-за чего производная равна нулю везде, где задана $\to$ о градиентных методах при поиске оптимального решения можно забыть
2. дерево решений не сможет экстраполировать зависимости за границы области значений обучающей выборки
3. дерево решений способно идеально приблизить обучающую выборку и ничего не выучить (переобучиться): дерево с одним объектом в каждом листе $\to$ надо не просто приближать обучающую выборку как можно лучше, но и стремиться оставлять дерево как можно более простым.

### Пример переобучения

Переобученное дерево - не ограничивали глубину:

![](https://yastatic.net/s3/ml-handbook/admin/3_5_74f1de3be9.png)

Нормально обученное дерево - ограничили глубину значением 3:

![](https://yastatic.net/s3/ml-handbook/admin/3_4_aa20f33d21.png)

## Обучение дерева

Обучение с учителем, имеем датасет $(X, y)$:
- $X = \{x_i^{(1)}, \cdots, x_i^{(D)}\}_{i=1}^N, \quad x_i^{(j)} \in \mathbb{R} \quad \forall i, j $
- $y = \{y_i\}_{i=1}^N, \quad y_i \in \mathbb{R} \quad \forall i$

Пусть так же задана функция потерь $L(f, X, y)$.

Наша задача — построить решающее дерево $f$, наилучшим образом предсказывающее целевую зависимость - минимизируя $L$. Но градиенты использовать нельзя! Как быть?

### Решающие пни

Давайте начнём с простого — научимся строить решающие пни, то есть решающие деревья глубины $1$:

$$B_{j,t}(x) = [x_i^{(j)} \leq t]$$

Эту задачу можно решить полным перебором: существует не более $(N-1)D$ предикатов такого вида - $j$ пробегает значения от $1$ до $D$, а всего значений порога $t$, при которых меняется значение предиката, может быть не более $N-1$:

![](https://yastatic.net/s3/ml-handbook/admin/3_7_f51986c1ae.png)

Искомое решение - $(j_{opt}, t_{opt}) = \arg\min\limits_{j,t} L(B_{j,t}, X, y)$.

Для каждого из $B_{j,t}$ нужно посчитать значение функции потерь - это еще $N$ операций. Итоговая сложность - $O(N^2D)$

Сложность неприятная, но не заоблачная. Проблема в том, что мы посчитали оптимум только для одной вершины, а что если их больше?

Кроме того, при построении дерева нужно учесть склонность к переобучению, поэтому хотим построить **оптимальное по качеству** на обучающей выборке дерево **минимальной глубины**.

Такая задача является $NP$-полной - проще говоря, кроме как полным перебором ее не решить :(

Как быть в такой ситуации? Есть два выхода:

1. **Искать не оптимальное решение, а просто достаточно хорошее.**
    
    Строить дерево с помощью жадного алгоритма - не искать всю структуру сразу, а строить дерево "этаж за этажом". Тогда в каждой внутренней вершине дерева будет решаться задача, схожая с задачей построения решающего пня.
    
    Для того чтобы этот подход хоть как-то работал, его придётся прокачать внушительным набором эвристик.

2. **Оптимизировать алгоритм полного перебора** — наивную версию алгоритма (перебор наборов возможных предикатов и порогов) можно ускорить и асимптотически, и в константу раз.

### Жадный алгоритм построения решающего дерева

Пусть $X$ — исходное множество объектов обучающей выборки, а $X_m$ — множество объектов, попавших в текущий лист (в самом начале), тогда жадный алгоритм условно можно описать следующим образом:

1. Создаём вершину $v$
2. Если выполнен **критерий остановки** $Stop(X_m)$
, то останавливаемся, объявляем эту вершину листом и ставим ей в соответствие **ответ** $Ans(X_m)$, после чего возвращаем её.
    
    Иначе: находим предикат (иногда ещё говорят сплит) $B_{j,t}$, который определит наилучшее разбиение текущего множества объектов $X_m$ на две подвыборки $X_l$ и $X_r$, максимизируя **критерий ветвления** $Branch(X_m, j, t)$.
3. Для $X_l$ и $X_r$ рекурсивно повторим процедуру.

Данный алгоритм содержит в себе вспомогательные функции, которые нужно выбрать так, чтобы итоговое дерево хороошо минимизировало $L$:

- $Ans(X_m)$ - вычисляет ответ для листа по попавшим в него объектам из обучающей выборки:
    - классификация - метка самого частого класса
    - регрессия - среднее, медиана, другая статистика
    - простая модель - можем здесь поместить другую модель!
- Критерий остановки $Stop(X_m)$ - нужно ли продолжать ветвление:
    - обычно - тривиальное правило: глубина, кол-во объектов в $X_m$, степень однородности объектов
- Критерий ветвления $Branch(X_m, j, t)$ - измеряет, насколько хорошо предлагаемое разделение. Оценивает, насколько улучшится метрика качества по сравнению с не-ветвлением. С ее помощью их всех возможных сплитов выбираем тот, что дает наибольшее улучшение.

Остановимся на критериях ветвление подробнее.

## Критерии ветвления

Пусть $c$ - ответ дерева ($c \in \mathbb{R}$ для регрессии, $c \in \mathbb{N}$ или $c \in \mathbb{R}^K$ при $\sum_{i=1}^K c_i = 1$ для классификации), а $L(y_i, c)$ - заданная функция потерь.

Ищем оптимальный сплит $X_m  = X_l \cup X_r$. Если бы текущая вершина была листом, то ее ответ был бы $c$, тогда значение функции потерь:

$$\frac{1}{|X_m|} \sum_{(x_i,y_i) \in X_m} L(y_i, c)$$

Оптимальное значение этой величины:

$$H(X_m) = \min\limits_{c \in Y} \frac{1}{|X_m|}\sum_{(x_i,y_i) \in X_m} L(y_i, c)$$

называют **информативностью** (**impurity**). Чем она ниже, тем лучше объекты в листе можно приблизить константой.

Для решающего пня получим:

$$\frac{1}{|X_m|} \big( \sum_{(x_i,y_i) \in X_l} L(y_i, c_l) + \sum_{(x_i,y_i) \in X_r} L(y_i, c_r) \big)$$

Тогда разность информативностей исходной вершины и решающего пня:

$$H(X_m) - \frac{|X_l|}{|X_m|} H(X_l) - \frac{|X_r|}{|X_m|} H(X_r)$$

Для симметрии можем домножить на $|X_m|$:

$$Branch(X_m, j, t) = |X_m| \cdot H(X_m) - |X_l| \cdot H(X_l) - |X_r| \cdot H(X_r)$$

Эта величина неотрицательна - разделив объекты, хуже не сделаем. Чем она больше, тем лучше предлагаемое разделение.

Теперь рассмотрим, какие будут критерии ветвления для типичных задач:

### Регрессия - MSE

При MSE - $L(y_i,c ) = (y_i - c)^2$, тогда информативность:

$$H(X_m) = \min\limits_{c \in Y} \frac{1}{|X_m|}\sum_{(x_i,y_i) \in X_m} (y_i - c)^2$$

Оптимальное предсказание константного классификатора для MSE - среднее арифметическое $c = \frac{\sum y_i}{|X_m|}$:

$$H(X_m) = \sum_{(x_i,y_i) \in X_m} \frac{(y_i - \bar{y})^2}{|X_m|}$$

При жадной минимизации MSE информативность - оценка дисперсии целевых величин для объектов, попавших в лист.

Тогда выбирать сплиты нужно так, чтобы сумма дисперсий в листьях была как можно меньше.

### Регрессия - MAE

При MAE - $L(y_i,c ) = |y_i - c|$, оптимальное предсказание - медиана:

$$H(X_m) = \sum_{(x_i,y_i) \in X_m} \frac{|y_i - MEDIAN(y)|}{|X_m|}$$

### Классификация - Misclassification Error

Пусть в задаче $K$ классов, $p_k$ - доля объектов класса $k$ в текущей вершине:

$$p_k = \frac{1}{|X_m|} \sum_{(x_i, y_i) \in X_m} \mathbb{I}[y_i = k]$$

Нам интересна доля верно угаданных классов, функция потерь - $L(y_i, c) = \mathbb{I}[y_i \neq c]$.

Пусть предсказание модели в листе - один класс, тогда информативность:

$$H(X_m) = \min\limits_{c \in Y} \frac{1}{|X_m|}\sum_{(x_i,y_i) \in X_m} \mathbb{I}[y_i \neq c]$$

Оптимальное предсказание - самый частый класс $k_*$, информативность получится:

$$H(X_m) = \frac{1}{|X_m|}\sum_{(x_i,y_i) \in X_m} \mathbb{I}[y_i \neq k_*] = 1 - p_{k_*} = 1 - \max\limits_{k \in K} p_k$$

### Классификация - Энтропия

Предсказываем вероятностное распределение классов $(c_1, \cdots, c_K)$, можем подойти как к задаче логистической регрессии - через максимизацию логарифма правдоподобия. Пусть в вершине предсказывается фиксированное распределение $c$ (независящее от $x_i$), тогда правдоподобие:

$$ P(y|x, c) = P(y|c) = \prod_{(x_i,y_i) \in X_m} P(y_i|c) = \prod_{(x_i,y_i) \in X_m} \prod_{k=1}^K c_k^{\mathbb{I}[y_i=k]}, $$

прологарифмируем и получим:

$$ H(X_m) = \min\limits_{\sum_k c_k = 1} \big( -\frac{1}{|X_m|} \sum_{(x_i,y_i) \in X_m} \sum_{k=1}^K \mathbb{I}[y_i=k] \log c_k \big) $$

Оптимальная оценка вероятностей $c_k$ - это $p_k$, т.е. доля попавших в лист объектов этого класса - выводится через условный минимум (лагранжиан).


Подставим вектор $c=(p_1, \cdots, p_K)$:

$$H(X_m) = - \sum_{k=1}^K p_k \log p_k$$

### Классификация - Критерий Джини

>**Внимание!**
>
> Не путать с индексом Джини из экономики! Это разные вещи!

Предсказание - распределение вероятности $(c_1, \cdots, c_K)$. Вместо логарифма правдоподобия в качестве критерия можно использовать метрику (по сути - MSE по вероятностям).

Тогда информативность:

$$ H(X_m) = \min\limits_{\sum_k c_k = 1} \frac{1}{|X_m|} \sum_{(x_i,y_i) \in X_m} \sum_{k=1}^K (c_k - \mathbb{I}[y_i=k])^2 $$

Оптимальное занчение этой метрики - выборочные оценки частот классов $(p_1, \cdots, p_K)$, $p_i = \frac{1}{|X_m|} \sum_{i} \mathbb{I}[y_i = k]$. Подставим в выражение выше, упростим и получим:

$$H(X_m) = \sum_{k=1}^K p_k(1-p_k) = 1 - \sum_{k=1}^K p_k^2 $$

### Сравнение

![](https://habrastorage.org/r/w1560/files/a88/bc3/e18/a88bc3e185b246e088a4382e212e4473.png)

## Неоптимальность критериев

Несмотря на все наши телодвиждения выше, жадный алгоритм не всегда позволяет найти самое оптимальное решение.

Простейший пример - задача XOR:

![](https://yastatic.net/s3/ml-handbook/admin/3_10_08f8ee6402.png)

Вне зависимости от того, что вы оптимизируете, жадный алгоритм не даст оптимального решения задачи XOR.

Но этим примером проблемы не исчерпываются. Бывают ситуации, когда оптимальное с точки зрения выбранной метрики дерево вы получите с критерием ветвления, построенным по другой метрике (например, MSE-критерий для MAE-задачи или Джини для misclassification error).

## Особенности работы с данными

### Категориальные признаки

На первый взгляд - все отлично, $x^{(i)} \in C$, при разбиении будет $C_m = C_l \cup C_r$, предикат вида $[x^{(i)} \in C_r]$. Это очень логично и естественно, но количество сплитов - $2^{M-1} -1$, перебирать очень долго.


Намного лучше упорядочить значения $c_m$ и работать как с числами.

### Работа с пропусками

Одна из приятных особенностей деревьев — способность обрабатывать пропуски в данных. Разберёмся, что при этом происходит на этапе обучения и на этапе применения дерева.

Пусть у нас есть некоторый признак $x^{(i)}$, значение которого пропущено у некоторых объектов, $X_m$ - множество объектов, пришедших в рассматриваемую вершину, $V_m$ — подмножество $X_m$, состоящее из объектов с пропущенным значением $x^{(i)}$.

В момент выбора сплитов по этому признаку мы будем просто игнорировать объекты из $V_m$, а когда сплит выбран, мы отправим их в оба поддерева, присвоив им веса $\frac{|X_l|}{|X_m|}$ для левого и $\frac{|X_r|}{|X_m|}$ для правого поддеревьев.

На этапе предсказания, если в вершину, в которой сплит идет по пропущенному признаку, пришел объект с пропущенным значением в этом признаке, то отправляем его в каждую из веток, получаем предсказания, взвешиваем с весами $\frac{|X_l|}{|X_m|}$ для левого и $\frac{|X_r|}{|X_m|}$ для правого поддеревьев.

## Регуляризация деревьев

Деревья легко переобучаются, процесс ветвления нужно ограничивать.

Основные способы:

- ограничение по максимальной глубине дерева;
- ограничение на минимальное количество объектов в листе;
- ограничение на максимальное количество листьев в дереве;
- требование, чтобы функционал качества $Branch$ при делении текущей подвыборки на две улучшался не менее чем на $s$ процентов.

Делать это можно на разных этапах работы алгоритма:

- во время построения дерева - **pre-pruning** или **early stopping**
- построить дерево жадно без ограничений, а затем провести стрижку (**pruning**) - удалить некоторые вершины из дерева так, чтобы итоговое качество упало не сильно, но дерево начало подходить под условия регуляризации. При этом качество стоит измерять на отдельной, отложенной выборке.

# Ансамбли

**Ансамблевые методы** - парадигма машинного обучения, в которой несколько моделей обучаются для решения одной и той же проблемы и объединяются для получения лучших результатов. Основная гипотеза - при правильном сочетании слабых моделей мы можем получить более точные и/или надежные модели.

Вводится понятие **_слабых учеников_** (или **_базовых моделей_**). Часто базовые модели работают сами по себе не так хорошо в связи с тем, для этого есть две причины:
- имеют высокое смещение - например, модели с низкой степенью свободы
- имеют слишком большой разброс, чтобы быть устойчивыми (например, модели с высокой степенью свободы).

**Идея ансамблевых методов** - попытаться уменьшить смещение и/или разброс таких слабых учеников, объединяя несколько из них вместе, чтобы создать **_сильного ученика_** (или **_модель ансамбля_**), который достигает лучших результатов.

## Математическое обоснование

Пусть решаем задачу регрессии и обучаем $n$ моделей, каждая из которых имеет ошибку $\varepsilon_i$. Будем считать, что все ошибки распределены по нормальному закону с нулевым средним $\mathbb{E}[\varepsilon_i]=0$, дисперсией $\mathbb{E}[\varepsilon_i^2]=\nu$ и ковариацией $\mathbb{E}[\varepsilon_i \varepsilon_j]=c$. В таком случае средняя ошибка предсказания ансамбля будет равна:

$$ \mathbb{E}[(\frac{1}{n}\sum_i\varepsilon_i)^2] = \frac{1}{n^2} \mathbb{E}[\sum_i (\varepsilon_i^2 + \sum_{j \ne i}\varepsilon_i \varepsilon_j)] = \frac{1}{n} \nu + \frac{n-1}{n} c $$

Рассмотрим два крайных случая:
- $c = \nu$ - ошибки разных моделей идеально скоррелированы, получаем, что квадрат ошибки никак не изменился
- $c = 0$ - ошибки разных моделей полностью независимы, получаем линейное уменьшение ошибки с ростом количества моделей в ансамбле

Отсюда можно сделать следующие выводы:
1. ансамблирование моделей с одинаковыми ошибками не уменьшает ошибку ансамбля
2. чтобы получить значительное уменьшение ошибки мы должны ансамблировать модели, в которых предсказания, а следовательно и ошибки, сильно отличаются

Для получения моделей с нескоррелированными предсказаниями, используются следующие подходы:
- обучить модели на разных поднаборах данных
- обучить модели на разных поднаборах признаках
- обучить модели с разной начальной инициализацией параметров
- обучить разные типы моделей модели

Небольшое забегание вперед:
- **Bootstrap** - статистический метод, заключающийся в генерации выборок размера $B$ (так называемых бутстрэп-выборок) из исходного датасета размера $L$ путем случайного выбора элементов с повторениями в каждом из наблюдений $B$.
- **Бэггинг** (**bagging**, **bootstrap aggregation**) - один из основных методов ансамблирования. Основная идея - используем набор отдельных обучающих выборок $X^i$, полученных из исходной выборки $X$ с помощью бутстрапа, для обучения набора базовых моделей $b_i(x)=b(x, X^i)$, а затем создаем из них ансамбль $a(x)$:
$$a(x)=\frac{1}{k} \sum_{i=1}^k b_i(x)$$


# Случайный лес

**Случайный лес** - ансамбль алгоритмов, где базовым алгоритмом является решающее дерево.

## Алгоритм построения

1. Строим $k$ деревьев. Для построения $i$-го дерева:
	1. Сначала, как в обычном бэггинге, из обучающей выборки $X$ выбирается с возвращением случайная подвыборка $X_i$ того же размера, что и $X$.
    2. В процессе обучения каждого дерева **в каждой вершине** случайно выбираются $n < N$ признаков, где $N$ — полное число признаков (**метод случайных подпространств, Random Subspace Method RSM**), и среди них ищется оптимальное разбиение. Такой приём как раз позволяет управлять степенью скоррелированности базовых алгоритмов.
2. Чтобы получить предсказание ансамбля на тестовом объекте, усредняем отдельные ответы деревьев (для регрессии) или берём самый популярный класс (для классификации).
3. Profit. Мы построили **Random Forest (случайный лес)** – комбинацию бэггинга и метода случайных подпространств над решающими деревьями.

## Гиперпараметры

### Глубина деревьев

Т.к. бэггинг позволяет снизить разброс, но не смещение, то стоит отдавать предпочтения алгоритмам с низким смещением. Такими алгоритмами являются глубокие деревья, которые хорошо переобучаются и могут запомнить подвыборку подробно. Поэтому предсказание на тестовом объекте будет сильнее меняться в зависимости от обучающей подвыборки, зато в среднем будет близко к истине (высокая дисперсия, низкое смещение).

**Вывод:** используем глубокие деревья.


### Количество признаков для отдельного дерева

Ограничивая число признаков, которые используются в обучении одного дерева, мы также управляем качеством случайного леса:
- Больше признаков - больше корреляция между деревьями, меньше чувствуется эффект от ансамблирования
- Меньше признаков, тем слабее сами деревья.

**Практическая рекомендация** – брать корень из числа всех признаков для классификации и треть признаков для регрессии.

### Количество деревьев в случайном лесе

Увеличение числа элементарных алгоритмов в ансамбле не меняет смещения и уменьшает разброс. Т.к. число признаков и варианты подвыборок, на которых строятся деревья в случайном лесе, ограничены, уменьшать разброс до бесконечности не получится. Поэтому имеет смысл **построить график ошибки** от числа деревьев и ограничить размер леса в тот момент, когда ошибка перестанет значимо уменьшаться.

Вторым практическим ограничением на количество деревьев может быть **время работы ансамбля**. Однако есть положительное свойство случайного леса: случайный лес можно строить и применять **параллельно**, что сокращает время работы, если у нас есть несколько процессоров. Но процессоров, скорее всего, всё же сильно меньше числа деревьев, а сами деревья обычно глубокие. Поэтому на большом числе деревьев Random Forest может работать дольше желаемого и количество деревьев можно сократить, немного пожертвовав качеством.

## Преимущества

1. Одна из лучших "универсальных" моделей - вместе с наивным баейсом, логистической регрессией
2. Работает очень эффективно с пропущенными признаками - потому что на некоторых деревьях вообще не используется выпавших признак
3. Есть классные модификации:
	- Extremely Randomized Trees - при разбиении признак вообще рандомно берем и для него ищем threshold
	- Isolation Forest - алгоритм поиска аномалий, можем поставить эту задачу деревьям
4. Позволяет использовать тренировочный датасет для валидации - **Out Of Bag (OOB) estimation** (можно использовать и в обычном бэггинге) - обучение базовых моделей делается на подвыборке тестовой выборки, поэтому некоторые данные оттуда они не видели никогда.